## Webtoon Background Transformation Guide: Using SDXL Image to Image
When creating webtoons, you often need various moods (weather, time of day, etc.) for the same location. Using SDXL Image to Image feature, you can create desired variations based on existing webtoon backgrounds. This notebook demonstrates loading a base image and transforming it into different moods.

### 1. Environment Setup
Import required libraries and configure Amazon Bedrock client.

In [ ]:
import base64
import io
import json
import os
from PIL import Image
import boto3

# Configure Amazon Bedrock client
bedrock_client = boto3.client('bedrock-runtime')

### 2. Base64 Image Conversion Implementation
Utility function to convert images to base64 format.

In [ ]:
def image_to_base64(img) -> str:
    """Convert local image file path to a base64 string for Amazon Bedrock"""
    if isinstance(img, str):
        if os.path.isfile(img):
            print(f"Reading image from file: {img}")
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    else:
        raise ValueError(f"Expected str (filename). Got {type(img)}")

### 3. Loading Base Webtoon Image
Load the base image from local storage.

In [ ]:
base_image_path = "data/base_img.png"
base_image_b64 = image_to_base64(base_image_path)

Let's see the base image. The example image shows a school building in clear weather in animation/webtoon style.

In [ ]:
base_image = Image.open(base_image_path)
base_image

### 4. Writing Transformation Prompts
Write prompts for weather transformation. This example changes the scene to rainy weather. Feel free to customize the prompts for different moods: sunny day, snowy night, foggy morning, or any atmosphere you want to create.

In [ ]:
prompt = "add rain effects, dark clouds, wet surfaces with reflections, school building in rain, puddles in school yard reflecting lights, rainy atmosphere, maintain webtoon style and clear lines"

negative_prompts = [
    "sunny", 
    "clear sky", 
    "blur", 
    "low quality",
    "people",
    "distorted architecture",
    "photorealistic"
]

### 5. Generate Image
Generate a transformed image with different atmosphere using SDXL model. you can refer to the [SDXL API docs](https://platform.stability.ai/docs/api-reference#tag/SDXL-1.0-and-SD1.6/operation/imageToImage) for more tips on how to use the different parameters:

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt, "weight": 1.0}]
        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
    ),
    "cfg_scale": 8,
    "init_image": base_image_b64,
    "seed": 321,
    "start_schedule": 0.7,
    "steps": 50,
    "style_preset": "anime",
    "clip_guidance_preset": "FAST_BLUE",
    "sampler": "K_DPMPP_2M"
})

modelId = "stability.stable-diffusion-xl-v1"
response = bedrock_client.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

In [ ]:
# Display result image
variant_image_b64 = response_body["artifacts"][0].get("base64")
variant_image = Image.open(io.BytesIO(base64.decodebytes(bytes(variant_image_b64, "utf-8"))))
variant_image